In [2]:
# Helper function for division with remainder
def divisionWithRemainder(a,b):
    return a // b, a % b

In [3]:
def extendedGCD(a,b):
    a, b = abs(a), abs(b) # Make sure a and b are positive
    
    r0, r1 = a, b # Set r0 = a and r1 = b
    u0, u1 = 1, 0 # Coefficients for a (u0 = 1, u1 = 0 for the first step)
    v0, v1 = 0, 1 # Coefficients for b (v0 = 0, v1 = 1 for the first step)
    
    while r1 != 0:
        # Perform division with remainder
        q, r2 = divisionWithRemainder(r0, r1)
        
        # Update r0, r1 for the next iteration
        r0, r1 = r1, r2
        
        # Update u0, u1 based on the division
        u0, u1 = u1, u0 - q * u1
        
        # Update v0, v1 based on the division
        v0, v1 = v1, v0 - q * v1
        
    # At the end, r0 contains the gcd, u0 and v0 are the coefficients for a, b
    return r0, u0, v0

In [4]:
def intToText(n):
    # Initialize an empty list to store the chars
    chars = []
    
    # Keep dividing n by 256 and collect remainders
    while n > 0:
        # Get remainder, this gives us a digit in base-256
        remainder = n % 256
        
        # Convert current digit into corresponding character using chr
        chars.append(chr(remainder))
        
        # Update n by didviding it by 256 for the next digit
        n = n // 256
        
    # Join together and return the list of chars at the end
    return ''.join(chars)

In [5]:
def textToInt(w):
    # Initialize n, this will hold final int result
    n = 0
    # Loop over each character in string w, along with its index i
    for i, char in enumerate(w):
        # Get the ascii value of current character using ord(char)
        # Multiply it by 256^i to get its value in base 256
        n += ord(char)*(256**i)
    # After processing all the characters, return the accumulated int n    
    return n

In [6]:
import random
def findPrime(lowerBound, upperBound):
    # Keep trying until we find a prime
    while True:
        # Pick a random number between lowerBound and upperBound
        n = random.randint(lowerBound, upperBound)
        
        # Check if the number is probably prime
        if probablyPrime(n):
            return n # Return the prime number found 'n'

In [7]:
import random
def probablyPrime(n):
    # Run the MillerRabin test 20 times with random base 'a'
    for _ in range(20):
        a = random.randint(2, n-1) # Random int between 2 and n-1
        # Check if current a is a MillerRabin witness for 'n'
        if MillerRabin(a, n):
            return False # If we find a witness, n is composite
    return True # If no witness is found after 20 tests, n is probably prime

In [8]:
def isElliptic(E, p):
    # Get A and B from the equation 
    A,B = E
    
    # Calculate ∆ = 4A^3 + 27B^2
    delta = (4*A**3 + 27*B**2) % p
    
    # Check if delta is nonzero mod p
    if delta != 0:
        return True
    else:
        return False  

In [9]:
def onCurve(P,E,p):
    # Check if P is point ar infinity (O)
    if P == "O":
        return True
    
    # Extract coordinates (x,y) and curve parameters (A, B)
    x, y = P
    A, B = E
    
    # Compute y^2 mod p
    lhs = (y**2) % p
    
    # Compute x^3 + AX + B mod p
    rhs = (x**3 + A*x + B) % p
    
    # Check if point satisfies elliptic curve equation mod p
    if lhs == rhs:
        return True
    else:
        return False

In [10]:
def modInverse(a, p):
    # Function to compute the modular inverse of a modulo p
    gcd, x, y = extendedGCD(a, p)
    if gcd != 1:
        return None # No inverse exists
    return x % p

In [11]:
def addPoints(P, Q, E, p):
    # Extract curve parameters
    A, B = E
    
    # Check if either P or Q is point at infinity:
    if P == "O":
        return Q
    
    if Q == "O":
        return P
    
    x1, y1 = P
    x2, y2 = Q
    
    # Point doubling
    if P == Q:
        # If y1 is 0, the point is at infinity
        if y1 == 0:
            return "O"
        
        # Get slope for point doubling
        num = (3 * x1**2 + A) % p
        den = (2 * y1) % p
        
        # Find modular inverse of denominator 
        inv = modInverse(den, p)
        if inv is None:
            return "O"  # No inverse exists, point is at infinity
        
        # Slope
        lam = (num * inv) % p
        
        # Compute x3 and y3
        x3 = (lam**2 - 2 * x1) % p
        y3 = (lam * (x1 - x3) - y1) % p
        return (x3, y3)
        
    # General case for adding 2 distinct primes
    # If x1 == x2, then P+Q is O
    if x1 == x2:
        return "O"
    
    # Slope for distinct primes
    num = (y2 - y1) % p
    den = (x2 - x1) % p

    # Find modular inverse of denominator
    inv = modInverse(den, p)
    if inv is None:
        return "O"
    
    # Slope
    lam = (num * inv) % p
    
    # Compute x3 and y3
    x3 = (lam**2 - x1 - x2) % p
    y3 = (lam * (x1 - x3) - y1) % p
    
    return (x3, y3)

In [12]:
def doubleAndAdd(P, n, E, p):      
    # Initialize the result as the point at infinity "O"
    R = "O"
    
    # Start with Q = P (the point we want to multiply by n)
    Q = P
    
    # While there are still bits left in the binary representation of n
    while n > 0:
        # If the current bit of n is 1, we add Q to R
        if n % 2 == 1:
            # Add R to Q
            R = addPoints(R, Q, E, p)
        # Double the point Q (this corresponds to squaring in fast exponentiation)
        Q = addPoints(Q, Q, E, p)
        
        # Shift n to the right by 1 (divide n by 2)
        n = n // 2
        
    # Return final result (nP)
    return R  

In [13]:
def fastPower(g, A, N):
    # Initialize a = g and b = 1
    a = g % N # Reduce g mod N first
    b = 1
    
    # Apply the algorithm while A > 0
    while A > 0:
        # If A is odd, multiply by a and take modulo N
        if A % 2 == 1:
            b = (b * a) % N
            
        # Square a and reduce A by half
        a = (a * a) % N
        A = A // 2
   
    # Return the result b
    return b 

In [14]:
def MillerRabin(a, n):
    # Factor n-1 into 2^k * m, where m is odd
    k = 0
    m = n-1
    while m % 2 == 0:
        m//= 2
        k += 1
        
    # Compute a^m mod n using fast exponentiation
    am = fastPower(a, m, n)
    
    # If a^m ≡ 1 mod n, then a is not a witness
    if am == 1:
        return False
    
    # Check for a^2^i * m ≡ -1 mod n for i = 0, ..., k-1
    for i in range(k):
        if am == n - 1:
            return False
        am = (am * am) % n # Compute a^2^i by succesive squaring
        
    # If we never found a^2^i * m ≡ -1 mod n, then a is a witness
    return True

In [15]:
def generateEllipticCurveAndPoint(p):
    
    # Pick a random a,b, A in the range [1, p-1]
    a = random.randint(1, p-1)
    b = random.randint(1, p-1)
    A = random.randint(1, p-1)
    
    # Solve for B using the equation: b^2 ≡ a^3 + A*a + B (mod p)
    # So, B ≡ b^2 - a^3 - A*a mod p
    lhs = (b**2) % p
    rhs = (a**3 + A*a) % p
    B = (lhs - rhs) % p
    
    # Form the elliptic curve and check if it's valid
    E = (A, B)
    if isElliptic(E, p) and onCurve((a,b), E, p):
        # Return the elliptic curve and the point on it
        return E, (a, b)

In [17]:
# Testing with small prime p = 7
p = 7
E, P = generateEllipticCurveAndPoint(p)
print("Prime p =", p)
print("Elliptic curve E:", E)
print("Point P:", P)
print("On curve?", onCurve(P, E, p))

Prime p = 7
Elliptic curve E: (2, 4)
Point P: (2, 4)
On curve? True


In [18]:
# Testing with 3-digit prime
p = findPrime(100, 999)
E, P = generateEllipticCurveAndPoint(p)
print("Prime p =", p)
print("Elliptic curve E:", E)
print("Point P:", P)
print("On curve?", onCurve(P, E, p))

Prime p = 587
Elliptic curve E: (271, 30)
Point P: (432, 512)
On curve? True


In [19]:
# Testing with a 50-bit prime
p = findPrime(2**49, 2**50-1)
E, P = generateEllipticCurveAndPoint(p)
print("Prime p =", p)
print("Elliptic curve E:", E)
print("Point P:", P)
print("On curve?", onCurve(P, E, p))

Prime p = 651524980992241
Elliptic curve E: (411278505214852, 487293750491844)
Point P: (591908890843294, 598764867336643)
On curve? True


In [20]:
 def MVParameterCreation(b):
    while True:
        # Generate a random b-bit prime
        lower = 2**(b-1)
        upper = 2**b-1
        p = findPrime(lower, upper)
        
        # Generate a curve and a point on it
        E, P = generateEllipticCurveAndPoint(p)
        
        # Make sure the point isnt the point at infinity and y != 0
        if P != "0" and P[1] != 0:
            return [E, P, p]

In [21]:
def MVKeyCreation(pubParams):
    E, P, p = pubParams
    
    # Choose a private key d randomly from [1, p-1]
    d = random.randint(1, p-1)
    
    # Compute the public key
    Q = doubleAndAdd(P, d, E, p)
    
    # Return the private and public keys
    return [d, Q]

In [22]:
def MVEncrypt(pubParams, m1, m2, publicKey):
        E, P, p = pubParams
        
        # Choose random key k
        k = random.randint(1, p-1)
        
        # Compute R = kp
        R = doubleAndAdd(P, k, E, p)
        
        # Compute S = kQ
        S = doubleAndAdd(publicKey, k, E, p)
        
        # Extract S_x and S_y
        Sx, Sy = S
        
        # Encrypt the messages
        c1 = (m1 * Sx) % p
        c2 = (m2 * Sy) % p
        
        # Return ciphertext
        return [R, c1, c2]

In [23]:
def MVDecrypt(pubParams, cipherText, privateKey):
    E, P, p = pubParams
    R, c1, c2 = cipherText
    
    # Compute shared secret S = d * R
    S = doubleAndAdd(R, privateKey, E, p)
    
    Sx, Sy = S
    
    # Compute modular inverse of Sx and Sy
    invSx = modInverse(Sx, p)
    invSy = modInverse(Sy, p)
    
    # Recover original messages
    m1 = (c1 * invSx) % p
    m2 = (c2 * invSy) % p
    
    return [m1, m2]

In [24]:
# Testing
# Generate public params and keys using a 32-bit prime
pubParams = MVParameterCreation(32)
E, P, p = pubParams
privKey, pubKey = MVKeyCreation(pubParams)

# Encrypt the given messages
m1 = 314159
m2 = 8675309
cipherText = MVEncrypt(pubParams, m1, m2, pubKey)

# Decrypt
decrypted = MVDecrypt(pubParams, cipherText, privKey)

# Output Results
print("Public parameters:")
print(" p =", p)
print(" E =", E)
print(" P =", P)
print("\nPrivate Key:", privKey)
print("Public Key:", pubKey)
print("\nCiphertext:", cipherText)
print("Decrypted:", decrypted)
print("Decryption successful?", decrypted == [m1, m2])

Public parameters:
 p = 2942770417
 E = (875124868, 492602333)
 P = (1249468676, 2410888042)

Private Key: 370711888
Public Key: (2786030146, 98304627)

Ciphertext: [(822687458, 2686618983), 1168155377, 1050751372]
Decrypted: [314159, 8675309]
Decryption successful? True


In [37]:
# Testing
# Generate public params with 512-bit prime
pubParams = MVParameterCreation(512)
E, P, p = pubParams
privKey, pubKey = MVKeyCreation(pubParams)

# Output
print("Prime p =", p)
print("Elliptic Curve E = (A,B) =", E)
print("Base point P =", P)
print("Public Key Q =", pubKey)

# Save private key
print("Private key =", privKey)

Prime p = 9214332594982606251698315098792825867470698854166486732332027227589597508687120927900406804613577025399446845025761605903347621092293527767370691936576121
Elliptic Curve E = (A,B) = (3223537829664451881486389031655264680649091007850911909318255836917101723971140146458043956195536180327129280759338141284694357607408557955129189753880841, 270467621103995365872251416765965538569218683396977984137056032876996401636528571793905097214104890904894235222779500830966967221264249401860192518645424)
Base point P = (5333784713640092362299081307067044265084527290827596656439798286513638564640138484541304404319230852308455365510843592475330469580726946751630331627630285, 4283060047572081555120532286527873714470322971185148523395581279639921451811081014593277543867663465206866885639502493054355597128016089867439540893552180)
Public Key Q = (55288662268751449035966615544035350893925354418738730223312373521408947036279576795212985827350705195613785345923685262654152921064389113710267837954835

In [31]:
# Testing with professor
# My parameters
p = 9214332594982606251698315098792825867470698854166486732332027227589597508687120927900406804613577025399446845025761605903347621092293527767370691936576121

E = ( 3223537829664451881486389031655264680649091007850911909318255836917101723971140146458043956195536180327129280759338141284694357607408557955129189753880841,   270467621103995365872251416765965538569218683396977984137056032876996401636528571793905097214104890904894235222779500830966967221264249401860192518645424
)

P = (
    5333784713640092362299081307067044265084527290827596656439798286513638564640138484541304404319230852308455365510843592475330469580726946751630331627630285, 4283060047572081555120532286527873714470322971185148523395581279639921451811081014593277543867663465206866885639502493054355597128016089867439540893552180
)

Q = (
    5528866226875144903596661554403535089392535441873873022331237352140894703627957679521298582735070519561378534592368526265415292106438911371026783795483566, 8334410466862130687691068415065516801466194282945568471570495912843769604764849889591003403597880203822886226815513412861660926792343759631064855681851504
)

privKey = 7369780417282635697880681440955862778800567661601350838671865843110212707456197002115315222341026341830766558493436321641272944874959744585711174436808769

# Ciphertext 
R = (   436361395139901298116095559513552246562468647860313556698809044237476929903909099136880556361640722257444151212627141846953391972224245504298698426651760, 2246585585131473942434398000156337221643902238956465801291229148204768036710964707281778172650031937182281666255855817138347193253472505473700800069720671
)

c1 = 883837613591533954070349825030856447044881031048910141225347145897373438726826619382326335168481738242303167074238700596831496577704870365107400524320721
c2 = 9092975451333870591771073847375877677603096142387642829759166729395942256152908882321848306921779995348723975790361979365615318702294117720528566442923144


# Prepare for decryption
pubParams = [E, P, p]
ciphertext = [R, c1, c2]

# Call decryption function
plaintext = MVDecrypt(pubParams, [R, c1, c2], privKey)

# Output decrypted integers
print("Decrypted integer values:", plaintext)

# Call intToText function:
decoded1 = intToText(plaintext[0])
decoded2 = intToText(plaintext[1])

print("Decoded messages:")
print("Message 1:", decoded1)
print("Message 2:", decoded2)

Decrypted integer values: [7107813380183178448447326576405815119478374137921657756504131751663576256021904297859212301933553025985588490954056, 131883654445667809153308170315003132777475078301638179236609313646649738627089661503132860043584590277973592909001380626173085047010357]
Decoded messages:
Message 1: Hello, Jack. Your secret numbers are 9876 and...
Message 2: 54321. Reply with the sum and product as the 2 messages.


In [29]:
# Replying to professor's message
# Given professor values:
p = 12318616371550721242199480619703283599374841926200314852758734816884090846038359547680260312862885907382768963411325139187362461239059264412702137770206997

A = 5149525586597486758114341175531429865118877380760554402968592395673657722881067251417551926855159505510322052358534052888218791537799097726937431812534080

B = 10318903348973275106992388709165258235529316451168320601101503863800629951895593677731895511366052339478340221844674306490829921855312165490735690580560555

xP = 11665005833384097379736577316574341217138446560864361172048216508044356295262768625202818803094421527248746727842878297378397806123628285879259928360276451

yP = 9556519663954537745904377144184015665914024953786951575246660649330912227051493788716304676895232264549607688192870163855030125922670125036925578675047857

xQ = 459397034323094612660117173305208624018649027739177037331656633764960371520850299177313850126613826298494111702222451834347940921347429286683543854214004

yQ = 513243265439784192451824887478427876304267549732548267624564983511902666176001403751206536910454041591070474339640893266317053138805472938146352021931325

# Rebuilding neccessary objects
E = (A, B)
P = (xP, yP)
Q = (xQ, yQ)

profParams = [E, P, p]

m1 = 64197  # Sum of the two secret numbers
m2 = 536474196  # Product of the two secret numbers

# Encrypt reply
encryptedMsg = MVEncrypt(profParams, m1, m2, Q)
print("[R, c1, c2]:", encryptedMsg)

[R, c1, c2]: [(6164315125707591775780089232121697176735265380968506346250307299154921789129102224132067516809762959338374600672459210847621870544570787621562853045297868, 7582972035566918482158826254539531508210613060161847251083082834360231993795359234369872753825354265953884060884624076220439475650736858073427272471039195), 7158729307565550078601509484298263377005715514952987204215022248780347109177603555418149293006958342753859925817836121654149674161444907261714353717810301, 2039217660801695504873621778194620998084486963058702303318802213694965280993829012851758993161915434815470867758157839146097345425614248938874993081588733]


In [47]:
def MVParameterCreationCompressed(b):
    while True:
        # Step 1: Generate a random b-bit prime
        lower = 2**(b - 1)
        upper = 2**b - 1
        p = findPrime(lower, upper)
        
        # Make sure Make sure p ≡ 3 mod 4
        if p % 4 != 3:
            continue
           
        # Generate curve and point
        E, P = generateEllipticCurveAndPoint(p)
        
        # Make sure the point isnt the point at infinity and y != 0
        if P != "0" and P[1] != 0:
            return [E, P, p]

In [46]:
def MVEncryptCompressed(pubParams, m1, m2, publicKey):
        E, P, p = pubParams
        
        # Choose random key k
        k = random.randint(1, p-1)
        
        # Compute R = kp
        R = doubleAndAdd(P, k, E, p)
        
        # Compute S = kQ
        S = doubleAndAdd(publicKey, k, E, p)
        
        xR, yR = R
        
        # Compress R to (xR, b)
        b = yR % 2 # Take parity bit of y
        
        # Compute sharefd secret S = kQ
        S = doubleAndAdd(publicKey, k, E, p)
        
        # Extract S_x and S_y
        Sx, Sy = S
        
        # Encrypt the messages
        c1 = (m1 * Sx) % p
        c2 = (m2 * Sy) % p
        
        # Return compressed ciphertext
        return [xR, b, c1, c2]

In [51]:
def MVDecryptCompressed(pubParams, cipherText, privateKey):
    E, P, p = pubParams
    A, B = E
    xR, b, c1, c2 = cipherText
    
    # Compute y^2 = x^3 + A*x + B mod p
    rhs = (xR**3 + A * xR + B) % p
    
    # Compute modular square root using fastPower (only works when p ≡ 3 mod 4)
    if p % 4 != 3:
        raise ValueError("This function requires p ≡ 3 mod 4")
        
    y = fastPower(rhs, (p+1) // 4, p)
    y_neg= (-y) % p
    
    # Use bit b to select the correct root
    yR = y if y % 2 == b else y_neg
    
    R = (xR, yR)
    
    # Compute shared secret S = d * R
    S = doubleAndAdd(R, privateKey, E, p)
    
    Sx, Sy = S
    
    # Compute modular inverse of Sx and Sy
    invSx = modInverse(Sx, p)
    invSy = modInverse(Sy, p)
    
    # Recover original messages
    m1 = (c1 * invSx) % p
    m2 = (c2 * invSy) % p
    
    return [m1, m2]

In [52]:
# Testing
# Generate public params and keys using a 32-bit prime
pubParams = MVParameterCreationCompressed(32)
E, P, p = pubParams
privKey, pubKey = MVKeyCreation(pubParams)

# Encrypt the given messages
m1 = 314159
m2 = 8675309
cipherText = MVEncryptCompressed(pubParams, m1, m2, pubKey)

# Decrypt
decrypted = MVDecryptCompressed(pubParams, cipherText, privKey)

# Output Results
print("Public parameters:")
print(" p =", p)
print(" E =", E)
print(" P =", P)
print("\nPrivate Key:", privKey)
print("Public Key:", pubKey)
print("\nCiphertext:", cipherText)
print("Decrypted:", decrypted)
print("Decryption successful?", decrypted == [m1, m2])

Public parameters:
 p = 3364498519
 E = (1510252936, 122997079)
 P = (228421337, 301960420)

Private Key: 121557325
Public Key: (1419086490, 1079260151)

Ciphertext: [1693120219, 0, 3151186665, 1688861434]
Decrypted: [314159, 8675309]
Decryption successful? True
